In [1]:
import json
import requests
import urllib.request
import time
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
df = pd.DataFrame()
index = 0
last_date = ''

# 수정 변수들
coinName = "BTC"
to_date = '2021-06-01'
num = 30
scale = '1'
if num == 0:
    type = 'days'
else:
    type = 'minutes/{}'.format(num)


def get_coin_data_url(coinName, type, scale, cnt=400) :
    addr = 'https://crix-api-endpoint.upbit.com/v1/crix/candles/'
    if type == 'minutes' :
        basic_url = addr + type + '/' + scale + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)
    else :
        basic_url = addr + type + '/' + '?code=CRIX.UPBIT.KRW-' + coinName +'&count='+str(cnt)

    return basic_url

basic_url = get_coin_data_url(coinName,type,scale)
url = basic_url

def get_upbit_data(url, last_date, to_date) :
    global df
    global index

    fail2GetData = False
    response = ''
    while True:
        try :
            response = requests.get(url)
        except Exception as e:
            time.sleep(20)
            continue
        if str(response) == '<Response [200]>':
            break
        time.sleep(10)
    if ( fail2GetData )  :
        exit()

    data = response.json()

    if index == 0:
        df = pd.DataFrame(data)
        date = ''
        date = data[len(data)-1]['candleDateTime']
        index +=1

    else:
        df_for = pd.DataFrame(data)
        df = df.append(df_for,ignore_index=True)
        date = ''
        date = data[len(data)-1]['candleDateTime']
    return date




In [2]:

while (1) :
    last_date = get_upbit_data(url, last_date, to_date)
    tmp1 = last_date.split('T')
    if tmp1[0]  < to_date :
        break
    tmp2 = tmp1[1].split('+')
    target_date = tmp1[0] + ' ' + tmp2[0]
    url = basic_url + '&to=' + target_date    #to=2019-11-27 04:01:00
    time.sleep(2)

idx_todate = df.index[(df.candleDateTimeKst == '{}T00:00:00+09:00'.format(to_date))].tolist()[0]+1
idx_todate

df = df.drop(df.index[idx_todate:])
df

,code,candleDateTime,candleDateTimeKst,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,candleAccTradePrice,timestamp,unit
0,CRIX.UPBIT.KRW-BTC,2021-06-13T18:00:00+00:00,2021-06-14T03:00:00+09:00,43323000.0,43490000.0,43265000.0,43470000.0,101.553482,4.405623e+09,1623608496503,30
1,CRIX.UPBIT.KRW-BTC,2021-06-13T17:30:00+00:00,2021-06-14T02:30:00+09:00,43212000.0,43483000.0,43111000.0,43324000.0,241.392565,1.045813e+10,1623607199291,30
2,CRIX.UPBIT.KRW-BTC,2021-06-13T17:00:00+00:00,2021-06-14T02:00:00+09:00,43040000.0,43220000.0,42800000.0,43212000.0,226.169257,9.724993e+09,1623605399460,30
3,CRIX.UPBIT.KRW-BTC,2021-06-13T16:30:00+00:00,2021-06-14T01:30:00+09:00,42860000.0,43217000.0,42812000.0,43038000.0,408.827124,1.759787e+10,1623603600160,30
4,CRIX.UPBIT.KRW-BTC,2021-06-13T16:00:00+00:00,2021-06-14T01:00:00+09:00,42034000.0,43101000.0,41929000.0,42860000.0,574.399061,2.448809e+10,1623601799854,30
...,...,...,...,...,...,...,...,...,...,...,...
623,CRIX.UPBIT.KRW-BTC,2021-05-31T17:00:00+00:00,2021-06-01T02:00:00+09:00,43750000.0,43781000.0,43344000.0,43455000.0,94.137519,4.097559e+09,1622482195149,30
624,CRIX.UPBIT.KRW-BTC,2021-05-31T16:30:00+00:00,2021-06-01T01:30:00+09:00,43732000.0,43900000.0,43580000.0,43706000.0,113.171244,4.945885e+09,1622480397168,30
625,CRIX.UPBIT.KRW-BTC,2021-05-31T16:00:00+00:00,2021-06-01T01:00:00+09:00,43452000.0,44235000.0,43331000.0,43732000.0,514.176556,2.254488e+10,1622478600042,30
626,CRIX.UPBIT.KRW-BTC,2021-05-31T15:30:00+00:00,2021-06-01T00:30:00+09:00,43382000.0,43600000.0,43200000.0,43452000.0,160.459124,6.963492e+09,1622476798899,30


In [3]:
df['Date'] = 0
df['hour'] = 0
df['minute'] = 0

In [4]:
df['Date'][0] = df['candleDateTimeKst'][0].split('T')
df['Date'][0][0]


'2021-06-14'

In [5]:
for i in range(len(df)):
    df['Date'] [i] = df['candleDateTimeKst'][i].split('T')
    df['hour'] [i] = df['Date'][i][1].split('+')[0]
    df['minute'] [i] = df['hour'][i].split(':')[1]
    df['hour'] [i] = df['hour'][i].split(':')[0]
    df['Date'] [i] = df['Date'][i][0]


In [6]:
df

,code,candleDateTime,candleDateTimeKst,openingPrice,highPrice,lowPrice,tradePrice,candleAccTradeVolume,candleAccTradePrice,timestamp,unit,Date,hour,minute
0,CRIX.UPBIT.KRW-BTC,2021-06-13T18:00:00+00:00,2021-06-14T03:00:00+09:00,43323000.0,43490000.0,43265000.0,43470000.0,101.553482,4.405623e+09,1623608496503,30,2021-06-14,03,0
1,CRIX.UPBIT.KRW-BTC,2021-06-13T17:30:00+00:00,2021-06-14T02:30:00+09:00,43212000.0,43483000.0,43111000.0,43324000.0,241.392565,1.045813e+10,1623607199291,30,2021-06-14,02,30
2,CRIX.UPBIT.KRW-BTC,2021-06-13T17:00:00+00:00,2021-06-14T02:00:00+09:00,43040000.0,43220000.0,42800000.0,43212000.0,226.169257,9.724993e+09,1623605399460,30,2021-06-14,02,0
3,CRIX.UPBIT.KRW-BTC,2021-06-13T16:30:00+00:00,2021-06-14T01:30:00+09:00,42860000.0,43217000.0,42812000.0,43038000.0,408.827124,1.759787e+10,1623603600160,30,2021-06-14,01,30
4,CRIX.UPBIT.KRW-BTC,2021-06-13T16:00:00+00:00,2021-06-14T01:00:00+09:00,42034000.0,43101000.0,41929000.0,42860000.0,574.399061,2.448809e+10,1623601799854,30,2021-06-14,01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,CRIX.UPBIT.KRW-BTC,2021-05-31T17:00:00+00:00,2021-06-01T02:00:00+09:00,43750000.0,43781000.0,43344000.0,43455000.0,94.137519,4.097559e+09,1622482195149,30,2021-06-01,02,0
624,CRIX.UPBIT.KRW-BTC,2021-05-31T16:30:00+00:00,2021-06-01T01:30:00+09:00,43732000.0,43900000.0,43580000.0,43706000.0,113.171244,4.945885e+09,1622480397168,30,2021-06-01,01,30
625,CRIX.UPBIT.KRW-BTC,2021-05-31T16:00:00+00:00,2021-06-01T01:00:00+09:00,43452000.0,44235000.0,43331000.0,43732000.0,514.176556,2.254488e+10,1622478600042,30,2021-06-01,01,0
626,CRIX.UPBIT.KRW-BTC,2021-05-31T15:30:00+00:00,2021-06-01T00:30:00+09:00,43382000.0,43600000.0,43200000.0,43452000.0,160.459124,6.963492e+09,1622476798899,30,2021-06-01,00,30


In [7]:

drops= ['Date','unit','candleAccTradePrice','openingPrice','code','candleDateTime','timestamp','candleDateTimeKst']
df=df.drop(drops,axis=1)
df=df[['hour','minute','tradePrice','highPrice','lowPrice','candleAccTradeVolume']]
df.head()

,hour,minute,tradePrice,highPrice,lowPrice,candleAccTradeVolume
0,03,0,43470000.0,43490000.0,43265000.0,101.553482
1,02,30,43324000.0,43483000.0,43111000.0,241.392565
2,02,0,43212000.0,43220000.0,42800000.0,226.169257
3,01,30,43038000.0,43217000.0,42812000.0,408.827124
4,01,0,42860000.0,43101000.0,41929000.0,574.399061


In [9]:
ups, downs = df['tradePrice'].loc[::-1].diff() ,df['tradePrice'].loc[::-1].diff()
ups[ups<0] =0
downs[downs>0] = 0
print(ups,downs)
period = 14 
ad = downs.abs().ewm(com = period-1, min_periods = period).mean() 
RS = au/ad 
RSI = pd.Series(100 - (100/(1+RS))) 
RSI = RSI.loc[::-1]
print(RSI)



627         NaN
626     52000.0
625    280000.0
624         0.0
623         0.0
         ...   
4      830000.0
3      178000.0
2      174000.0
1      112000.0
0      146000.0
Name: tradePrice, Length: 628, dtype: float64 627         NaN
626         0.0
625         0.0
624    -26000.0
623   -251000.0
         ...   
4           0.0
3           0.0
2           0.0
1           0.0
0           0.0
Name: tradePrice, Length: 628, dtype: float64


In [13]:
period = 14 
au = ups.ewm(com = period-1, min_periods = period).mean()
ad = downs.abs().ewm(com = period-1, min_periods = period).mean() 
RS = au/ad 
RSI = pd.Series(100 - (100/(1+RS))) 
RSI = RSI.loc[::-1]
print(RSI)


0      80.092584
1      78.817584
2      77.805001
3      76.161049
4      74.356820
         ...    
623          NaN
624          NaN
625          NaN
626          NaN
627          NaN
Name: tradePrice, Length: 628, dtype: float64
